In [ ]:
import os
os.environ['AWS_PROFILE'] = 'admin'
os.environ['HAVEN_DATABASE'] = 'haven'

import numpy as np
import pandas as pd
from scipy.stats import norm
from random import sample
import h3
from suntimes import SunTimes
from datetime import datetime
import ephem

from mirrorverse.utils import read_data_w_cache
from haven.db import write_data

In [ ]:
data = read_data_w_cache(
    'select * from chinook_depth_decisions'
)
print(data.shape)
data.head()

In [ ]:
data['lat'] = data['h3_index'].apply(lambda h: h3.h3_to_geo(h)[0])
data['lon'] = data['h3_index'].apply(lambda h: h3.h3_to_geo(h)[1])
data.head()

In [ ]:
def sunset(row):
    suntime = SunTimes(longitude=row['lon'], latitude=row['lat'], altitude=0)
    return suntime.setwhere(row['time'], "UTC")

def sunrise(row):
    suntime = SunTimes(longitude=row['lon'], latitude=row['lat'], altitude=0)
    return suntime.risewhere(row['time'], "UTC")

data['sunrise'] = data.apply(sunrise, axis=1).dt.time
data['sunset'] = data.apply(sunset, axis=1).dt.time
data['time'] = pd.to_datetime(data['epoch'], unit='s').dt.time
data.head()

In [ ]:
def convert_sun_to_radians(row):
    sunrise = datetime.combine(datetime.today(), row['sunrise'])
    sunset = datetime.combine(datetime.today(), row['sunset'])
    time = datetime.combine(datetime.today(), row['time'])

    sun_is_up = False
    percent_through = 0
    if time >= sunrise and time < sunset:
        sun_is_up = True
        percent_through = (time - sunrise).total_seconds() / (sunset - sunrise).total_seconds()
    elif time >= sunset and time < sunrise:
        sun_is_up = False
        percent_through = (time - sunset).total_seconds() / (sunrise - sunset).total_seconds()
    elif sunrise > sunset:
        sun_is_up = True
        total_duration = 24 * 60 * 60 - (sunrise - sunset).total_seconds() 
        if time >= sunrise:
            duration = (time - sunrise).total_seconds()
        else:
            duration = total_duration - (sunset - time).total_seconds()
        percent_through = duration / total_duration
    else:
        sun_is_up = False
        total_duration = 24 * 60 * 60 - (sunset - sunrise).total_seconds() 
        if time >= sunset:
            duration = (time - sunset).total_seconds()
        else:
            duration = total_duration - (sunrise - time).total_seconds()
        percent_through = duration / total_duration
    
    return percent_through * np.pi + (1 - sun_is_up) * np.pi


data['sun_radians'] = data.apply(lambda r: convert_sun_to_radians(r), axis=1)
data.head()

In [ ]:
data['cos_sun'] = np.cos(data['sun_radians'])
data['sin_sun'] = np.sin(data['sun_radians'])
data.head()

In [ ]:
def convert_moon_to_radians(date):
    edate = ephem.Date(date)
    nnm = ephem.next_new_moon(date)
    pnm = ephem.previous_new_moon(date)
    if nnm == edate or pnm == edate:
        percent_through = 0
    else:
        percent_through = (edate - pnm) / (nnm - pnm)
    return np.pi * 2 * percent_through


data['time'] = pd.to_datetime(data['epoch'], unit='s').dt.date
data['moon_radians'] = data['time'].apply(convert_moon_to_radians)
data.head()

In [ ]:
data['cos_moon'] = np.cos(data['moon_radians'])
data['sin_moon'] = np.sin(data['moon_radians'])
data.head()

In [ ]:
from datetime import date

def convert_orbit_to_radians(_date):
    return _date.timetuple().tm_yday / date(_date.year, 12, 31).timetuple().tm_yday * np.pi * 2

data['orbit_radians'] = data['time'].apply(convert_orbit_to_radians)
data.head()

In [ ]:
data['cos_orbit'] = np.cos(data['orbit_radians'])
data['sin_orbit'] = np.sin(data['orbit_radians'])
data.head()

In [11]:
del data['sunrise']
del data['sunset']

In [12]:
write_data(
    data, 'chinook_depth_decisions_w_time', ['_train']
)